### vavrinec:

- PD trigger 15v/5v -> step-up 15-18V -> 2s bat charger -> bat_protection_board gut?
- read voltage adc gut
- baterky (7-8.4V) -> STEPUP +15V -> INVERT -15V

# IN PROGRESS:
- RGB lasershow
  - TODO:
    - deltatime - always project the whole frame, but also always pick what frame to project based on what time it is (frames[(time - start)(seconds) * fps])

## TODO:
### misc:
- add batery indicators 
  - 4led https://www.gme.cz/v/1508436/led-indikator-stavu-li-ion-pol-baterie-4s <!-- https://pajenicko.cz/indikator-stavu-kapacity-lipol-liion-baterie-4s -->
  - voltage https://www.postavrobota.cz/LED-Indikator-stavu-kapacity-baterie-do-panelu-4S-li-ion-li-po-s-voltmetrem-d655.htm
  - nejlip adc na rpi
### sw:
#### UI/lasershow:
- pwm brigtness for leds
- select which leds are active
- seek

#### install:
- zeptat se na moznosti rozbiti systemu a instalaci napr dma a hostapd uz na zacatku

#### misc:
- rewrite wifi_manager to change what configs services use instead of changing configs themselves
- sockets in wifi_man
- sockets use envelopes to separate wifiman and lasershow -> mention in thesis
- sockets in web_ui, discord_bot
- rewrite discord_bot
### hw:
- check galvo current +15v-15v
- check cooler fan current 12V (stepup)

## thesis uvod
laser scanning se využívá v různých oborech víc a víc, cílem me práce je umožnit začínajícím kutilum se seznámit s touto technologií a rozvíjet se s ní

### menu.hpp

  // prusa style ui with oled display and rotary encoder - use ZMQ sockets for communication with lasershow.cpp


### discord help command

### wifi manager
sockets

### web-ui
sockets


## hw_notes:
- make attiny into a pwm generator (could it be an adc asw?)
  - https://forum.allaboutcircuits.com/threads/spi-i2c-pwm-generator.11959/
  - https://medium.com/jungletronics/fast-rainbow-circuit-f0dddd5ee28 (https://github.com/giljr/Ardu_Serie/blob/master/_30_Fast_Rainbow/_30_arduSerie_ATTiny85_three_PWM_output/_30_arduSerie_ATTiny85_three_PWM_output.ino)

## sw_notes:
lasershow receives shutdown message and cleans up laser - turn off relay
  also measure battery voltage

//how do i debounce well??

## WATCH-OUT
hostapd
  - The previously supported configuration setting DAEMON_CONF in
  /etc/default/hostapd is deprecated and its support will be removed.

  - SCRIPT DEPRECATION WARNING                    
  This script, located at https://deb.nodesource.com/setup_X, used to
  install Node.js is deprecated now and will eventually be made inactive.
  Please visit the NodeSource distributions Github and follow the
  instructions to migrate your repo.
  https://github.com/nodesource/distributions
  The NodeSource Node.js Linux distributions GitHub repository contains
  information about which versions of Node.js and which Linux distributions
  are supported and how to install it.
  https://github.com/nodesource/distributions
  https://github.com/nodesource/distributions#debian-versions


In [1]:
total = 0

filament = 500
# max 500czk za jeden - maybe 2x  
filament *= 2
total += filament

RPi = 2200 # RPi 5 8GB (wow, banger) https://rpishop.cz/raspberry-pi-5/6498-raspberry-pi-5-8gb-ram.html
total += RPi

galva = 2200
total += galva

laser = 1800
total += laser

signal = 0
signal += 2* 50 # amps - 30-50czk (tl082 / smaller range)
total += signal

napajeni = 0
# baterky
napajeni += 4*275 # 4-6x (podle toho, kolik se jich vleze) 18650 idk kolik stoji
# 4000mAh 275kc https://www.abaterie.cz/baterie-samsung-21700-inr21700-40t-li-ion-3-6-v-4000-mah/
# 2x 18650 - 6000mAh 425kc https://www.abaterie.cz/baterie-samsung-2xinr18650-30q-6000mah-20a/

napajeni += 2*22 # drzak na baterky https://www.laskakit.cz/bateriovy-box-2x18650-dratove-vyvody/

#ochrana
napajeni += 58 #https://www.laskakit.cz/ochrana-li-ion-baterie-2s-20a/

# nabijeni
napajeni += 48 # usb-c PD trigger board https://www.laskakit.cz/usb-c-pd-qc-prepinac-napajeciho-napeti/
napajeni += 38 # 4A step-up (->15-18v) https://www.laskakit.cz/step-up-boost-menic-s-xl6009--modra/

# napeti 7-8.4V->
napajeni += 48 # 5V 3A RPi https://www.laskakit.cz/step-down-menic-s-xl4005/
napajeni += 38 # 15V 1-1.5A galva (not sure, jakej stepup (https://www.laskakit.cz/step-up-boost-menic-s-mt3608/ / https://www.laskakit.cz/step-up-boost-menic-s-xl6009--modra/))
napajeni += 18 # 12V 1A laser https://www.laskakit.cz/step-up-boost-menic-s-mt3608/
#^^ pokud napajim i vetrak, radsi vetsi stepup (https://www.laskakit.cz/step-up-boost-menic-s-xl6009--modra/)
napajeni += 38 # https://www.laskakit.cz/1-kanal-5v-rele-modul--low-level--250vac-10a/
# bat adc + pwm gen
napajeni += 0 # attiny a rpi uz mam, ale mozna by bylo elegantnejsi reseni

# invert 15V
# napajeni += 88 # +-15V boost https://www.laskakit.cz/step-up-boost-menic-s-xl6007-15v-zdroj-symetrickeho-napeti/

napajeni += 70 # 65kc balikovna

total += napajeni

oled = 160 # https://www.laskakit.cz/20x4-lcd-displej-2004-modry-i2c-prevodnik/
total += oled

print ("mone: " + str(total))

mone: 8916


In [3]:
Vbat = 3.7
Vmax = Vbat*2
Imax = 20

Pmax = Vmax * Imax

print ("max power: " + str(Pmax) + "W")

P = 0
# 3A RPi
P += 5 * 3 # 5V
# -15V boost -> signal galva

# 1A galva
P += 1 * 15 # 15V
# 1A laser
P += 1 * 12 # 12V

print ("needed power: " + str(P) + "W")

max power: 148.0W
needed power: 42W
